# Prepare Data
- [wikitext-103 dataset](https://www.salesforce.com/products/einstein/ai-research/the-wikitext-dependency-language-modeling-dataset/)
- wikitest 이름 바꾸기 ㅡㅡ 뒤질라고

In [18]:
import scipy.sparse as sparse

In [14]:
import os
import shutil
import re
import time

import nltk
from nltk.stem import WordNetLemmatizer

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

import scipy.sparse as sparse

In [3]:
def check_create_dir(dir):
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.mkdir(dir)

In [4]:
# create directory for data
dataset = "wikitest-103"
current_dir = os.getcwd()

data_dir = os.path.join(current_dir, "data")
if not os.path.exists(data_dir):
    print('Creating directory:', data_dir)
    os.mkdir(data_dir)
data_dir = os.path.join(current_dir, "data", dataset)
check_create_dir(data_dir)
os.chdir(data_dir)
print('Current directory: ', os.getcwd())

Current directory:  C:\workspace\W-LDA\data\wikitest-103


In [5]:
# parse into documents
def is_document_start(line):
    if len(line) < 4:
        return False
    if line[0] is '=' and line[-1] is '=':
        if line[2] is not '=':
            return True
        else:
            return False
    else:
        return False


def token_list_per_doc(input_dir, token_file):
    lines_list = []
    line_prev = ''
    prev_line_start_doc = False
    with open(os.path.join(input_dir, token_file), 'r', encoding='utf-8') as f:
        for l in f:
            line = l.strip()
            if prev_line_start_doc and line:
                # the previous line should not have been start of a document!
                lines_list.pop()
                lines_list[-1] = lines_list[-1] + ' ' + line_prev

            if line:
                if is_document_start(line) and not line_prev:
                    lines_list.append(line)
                    prev_line_start_doc = True
                else:
                    lines_list[-1] = lines_list[-1] + ' ' + line
                    prev_line_start_doc = False
            else:
                prev_line_start_doc = False
            line_prev = line

    print("{} documents parsed!".format(len(lines_list)))
    return lines_list

In [7]:
input_dir = os.path.join(data_dir, dataset)
train_file = 'wiki.train.tokens'
val_file = 'wiki.valid.tokens'
test_file = 'wiki.test.tokens'
train_doc_list = token_list_per_doc(input_dir, train_file)
val_doc_list = token_list_per_doc(input_dir, val_file)
test_doc_list = token_list_per_doc(input_dir, test_file)

28472 documents parsed!
60 documents parsed!
60 documents parsed!


In [11]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jinma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
class LemmaTokenizer(object):
    
    def __init__(self):
        self.wnl = WordNetLemmatizer()

    def __call__(self, doc):
        return [
            self.wnl.lemmatize(t) for t in doc.split() 
            if (len(t) >= 2 and re.match("[a-z].*", t) and 
                re.match(token_pattern, t))
        ]

In [16]:
token_pattern = re.compile(r"(?u)\b\w\w+\b")

In [17]:
print('Lemmatizing and counting, this may take a few minutes...')
start_time = time.time()
vectorizer = CountVectorizer(input='content', analyzer='word', 
                             stop_words='english', tokenizer=LemmaTokenizer(), 
                             max_df=0.8, min_df=3, max_features=20000)

train_vectors = vectorizer.fit_transform(train_doc_list)
val_vectors = vectorizer.transform(val_doc_list)
test_vectors = vectorizer.transform(test_doc_list)

vocab_list = vectorizer.get_feature_names()
vocab_size = len(vocab_list)
print('vocab size:', vocab_size)
print('Done. Time elapsed: {:.2f}s'.format(time.time() - start_time))

Lemmatizing and counting, this may take a few minutes...


C:\Users\jinma\AppData\Local\Continuum\anaconda3\envs\basic\lib\site-packages\sklearn\feature_extraction\text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


vocab size: 20000
Done. Time elapsed: 401.46s


In [19]:
def shuffle_and_dtype(vectors):
    idx = np.arange(vectors.shape[0])
    np.random.shuffle(idx)
    vectors = vectors[idx]
    vectors = sparse.csr_matrix(vectors, dtype=np.float32)
    print(type(vectors), vectors.dtype)
    return vectors

In [20]:
train_vectors = shuffle_and_dtype(train_vectors)
val_vectors = shuffle_and_dtype(val_vectors)
test_vectors = shuffle_and_dtype(test_vectors)

with open('vocab.txt', 'w', encoding='utf-8') as f:
    for item in vocab_list:
        f.write(item+'\n')

sparse.save_npz('wikitext-103_tra.csr.npz', train_vectors)
sparse.save_npz('wikitext-103_val.csr.npz', val_vectors)
sparse.save_npz('wikitext-103_test.csr.npz', test_vectors)

<class 'scipy.sparse.csr.csr_matrix'> float32
<class 'scipy.sparse.csr.csr_matrix'> float32
<class 'scipy.sparse.csr.csr_matrix'> float32
